In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

root_dir = "../../"
sys.path.append(root_dir)

from utils import models, location_performance_t

data_dir = os.path.join(root_dir, "data")
rsna_dir = os.path.join(data_dir, "RSNA")
prediction_dir = os.path.join(rsna_dir, "predictions")
figure_dir = os.path.join(root_dir, "figures", "exam_level")
os.makedirs(figure_dir, exist_ok=True)

sns.set_theme()
sns.set_context("paper", font_scale=1.5)

In [ ]:
def exam_level_f1(cutoff):
    cutoff_method, cutoff_name = cutoff

    df = []
    for model_title, model, weak_supervision in tqdm(models):
        prediction_df = pd.read_pickle(
            os.path.join(prediction_dir, model, "predictions")
        )
        prediction_df.set_index("series_idx", inplace=True)

        for min_length in range(1, 7):
            _df = location_performance_t(
                model_title,
                prediction_df,
                weak_supervision,
                cutoff=cutoff_method,
                min_length=min_length,
            )
            df.append(_df)
    df = pd.concat(df)

    _, ax = plt.subplots(figsize=(16 / 2, 9 / 2))
    sns.lineplot(data=df, x="min_length", y="f1", hue="est_name", ax=ax)
    ax.set_xlabel("Minimal sequence length")
    ax.set_ylabel(r"$f_1$ score")
    ax.set_xticks(range(1, 7))
    ax.legend(title="", loc="upper left", bbox_to_anchor=(1, 1))
    ax.set_title(cutoff_name)

    plt.savefig(
        os.path.join(figure_dir, f"RSNA_f1_{cutoff_method}_min_seq_length.jpg"),
        bbox_inches="tight",
    )
    plt.savefig(
        os.path.join(figure_dir, f"RSNA_f1_{cutoff_method}_min_seq_length.pdf"),
        bbox_inches="tight",
    )
    plt.show()
    plt.close()

In [ ]:
cutoffs = [("youden", r"Youden's $J$ statistic"), ("d", r"Distance to $(0,1)$ point")]
for cutoff in cutoffs:
    exam_level_f1(cutoff)